Thực hiện cào dữ liệu với các thông tin sau:

- Company Link: Đường dẫn của công ty
- Company Name: Tên công ty
- Company Location: Địa điểm của công ty
- Company Scale: Quy mô công ty
- Company Type: Loại hình hoạt động
- Company Website: Website của công ty
- Followers: Người theo dõi
- Company Intro: Giới thiệu của công ty
- Job Name: Tên công việc
- Link : Đường dẫn của job.
- Location: Địa điểm.
- Update Date: Ngày up CV.
- Deadline: Hạn nộp CV.
- Salary: Lương.
- Experience: Kinh nghiệm.
- Rank: Cấp bậc.
- Briefcase: Loại ngành nghề
- Job type: Hình thức làm việc.
- Welfare: Phúc lợi.
- Job Description: Mô tả công việc.
- Job Requirement: Yêu cầu công việc.
- Job Location: Địa điểm làm việc.
- Other Information: Thông tin khác.
- Job Tags: Job Tags

_executable_path_ mọi người tải về chromedriver.exe phù hợp với phiên bản của Chrome đang xài rồi dán đường dẫn chứa chromedriver.exe để cào dữ liệu. Còn trình duyệt khác thì không rõ.

In [3]:
# Necessary Packages 
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [4]:
BASE_URL = "https://careerviet.vn/viec-lam/"

# Set up the Chrome Options
executable_path = "D:/KHTN/datascienceintro/chromedriver-win64/chromedriver-win64/chromedriver.exe"

In [22]:
def crawlJobLinks(pageStart: int,pageEnd:int) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(executable_path=executable_path,options=chrome_options)
    job_links_list=[]
    for page in range(pageStart,pageEnd+1):
        url= f"{BASE_URL}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR,'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))
        
    driver.quit()
    return job_links_list

In [23]:
def getInfoJob(job_url) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(executable_path=executable_path,options=chrome_options)
    
    driver.get(job_url)
    time.sleep(random.randint(5, 10))
    
    #Tên công việc
    try:
        job_name_element=driver.find_element(By.CSS_SELECTOR,".job-desc h1.title")
        job_name=job_name_element.text
    except:
        job_name="Not Found"
    #Địa điểm
    try:
        location_element=driver.find_element(By.CSS_SELECTOR,".map p a")
        location=location_element.text
    except:
        location="Not Found"

    #Ngày cập nhật
    try:
        update_date=driver.find_element(By.XPATH,"//strong[contains(text(),'Ngày cập nhật')]/following-sibling::p").text
    except:
        update_date="Not Found"

    #Loại ngành nghề
    try:
        li_briefcase_element = driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-briefcase)")
        a_briefcase_elements=li_briefcase_element.find_elements(By.CSS_SELECTOR,"a")
        list_briefcase=""
        for link in a_briefcase_elements:
            list_briefcase+=link.text+", "   
        list_briefcase=list_briefcase.rstrip(", ")
    except:
        list_briefcase="Not Found"

    #Hình thức
    try:
        job_type=driver.find_element(By.XPATH,"//strong[contains(text(),'Hình thức')]/following-sibling::p").text
    except:
        job_type="Not Found"

    #Lương
    try:
        salary=driver.find_element(By.XPATH,"//strong[contains(text(),'Lương')]/following-sibling::p").text
    except:
        salary="Not Found"

    #Kinh nghiệm
    try:
        experience=driver.find_element(By.XPATH,"//strong[contains(text(),'Kinh nghiệm')]/following-sibling::p").text
    except:
        experience="Not Found"

    #Cấp bậc
    try:
        rank=driver.find_element(By.XPATH,"//strong[contains(text(),'Cấp bậc')]/following-sibling::p").text
    except:
        rank="Not Found"

    #Deadline
    try:
        deadline=driver.find_element(By.XPATH,"//strong[contains(text(),'Hết hạn nộp')]/following-sibling::p").text
    except:
        deadline="Not Found"

    #Phúc lợi
    try:
        li_elements=driver.find_elements(By.CSS_SELECTOR,".welfare-list > li")
        list_welfare=""
        for li in li_elements:
            list_welfare+=li.text+", "
        list_welfare=list_welfare.rstrip(", ")
    except:
        job_description="Not Found"

    #Mô tả công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet")
        job_description=h2_element.text
    except:
        job_description="Not Found"


    #Yêu cầu công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet + .reset-bullet")
        job_requirement=h2_element.text
    except:
        job_requirement="Not Found"

    #Địa điểm làm việc
    try:
        place_element=driver.find_element(By.CSS_SELECTOR,".place-name .place")
        job_location=place_element.text
    except:
        job_location="Not Found"

    #Thông tin khác
    try:
        other_info_element=driver.find_element(By.CSS_SELECTOR,".content_fck")
        other_info=other_info_element.text
    except:
        other_info="Not Found"

    #Thông tin về link công ty
    try:
        company_element=driver.find_element(By.CSS_SELECTOR,"#tabs-job-company a")
        company_url=company_element.get_attribute("href")
    except:
        company_url="Not Found"
    
    #Job tags
    try:
        job_tags_elements=driver.find_elements(By.CSS_SELECTOR,".job-tags li")
        job_tags=""
        for tag in job_tags_elements:
            job_tags+=tag.text+", "
        job_tags=job_tags.rstrip(", ")
    except:
        job_tags="Not Found"

    driver.get(company_url)
    time.sleep(random.randint(5, 10))

    #Tên công ty
    try:
        company_name_element=driver.find_element(By.CSS_SELECTOR,".company-info .content .name")
        company_name=company_name_element.text
    except:
        company_name="Not Found"

    #Địa điểm cụ thể của công ty
    try:
        company_location_element=driver.find_element(By.CSS_SELECTOR,".company-info .content p")
        company_location=company_location_element.text
    except:
        company_location="Not Found"
    
    #Quy mô công ty
    try:
        company_scale_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-account-supervisor")
        company_scale=company_scale_element.text
        company_scale=company_scale.replace("Qui mô công ty: ","")
    except:
        company_scale="Not Found"

    #Loại hình hoạt động
    try:
        company_type_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-gavel)")
        company_type=company_type_element.text
        company_type=company_type.replace("Loại hình hoạt động: ","")
    except:
        company_type="Not Found"
    
    #Đường dẫn công ty
    try:
        company_website_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-link)")
        company_website=company_website_element.text
        company_website=company_website.replace("Website: ","")        
    except:
        company_website="Not Found"

    #Số lượng người theo dõi
    try:
        follow_element=driver.find_element(By.CSS_SELECTOR,"#countFollowers")
        follower=follow_element.text
    except:
        follower="Not Found"

    #Giới thiệu công ty
    try:
        company_intro_element=driver.find_element(By.CSS_SELECTOR,".intro-section .main-text")
        company_intro=company_intro_element.text
    except:
        company_intro="Not Found"
    

    driver.quit()


    job_information=[company_url,company_name,company_location,company_scale,company_type,company_website,follower,company_intro,job_name,job_url,location,update_date,deadline,salary,experience,rank,job_type,list_briefcase,list_welfare,job_description,job_requirement,job_location,other_info,job_tags]

    return job_information 

In [24]:
def crawlJobInformation(job_urls) -> pd.DataFrame:
    job_information_list=[]
    for job_url in job_urls:
        job_information=getInfoJob(job_url)
        job_information_list.append(job_information)
    df=pd.DataFrame(job_information_list,columns=["Company Link","Company Name","Company Location","Company Scale","Company Type","Company Website","Followers","Company Intro","Job Name","Link","Location","Update Date","Deadline","Salary","Experience","Rank","Job Type","Briefcase","Welfare","Job Description","Job Requirement","Job Location","Other Information","Job Tags"])
    return df

In [ ]:
##Mẫu

,Company Link,Company Name,Company Location,Company Scale,Company Type,Company Website,Followers,Company Intro,Job Name,Link,...,Experience,Rank,Job Type,Briefcase,Welfare,Job Description,Job Requirement,Job Location,Other Information,Job Tags
0,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,CÔNG TY CỔ PHẦN TẬP ĐOÀN Y DƯỢC EBC,"106 Bát Nàn, Phường Bình Trưng Tây, Quận 2, Hồ...",100-499,Cổ phần,https://ebcpharma.vn,913,"Khởi nguồn từ nhu cầu thị trường, Tập đoàn Y d...",Trưởng phòng Hành chính Nhân sự,https://careerviet.vn/vi/tim-viec-lam/truong-p...,...,5 - 10 Năm,Quản lý,Nhân viên chính thức,"Hành chính / Thư ký, Nhân sự, Sản xuất / Vận h...","Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồn...",MÔ TẢ CÔNG VIỆC\n- Điều hành và quản lý hoạt đ...,YÊU CẦU CÔNG VIỆC\n- Có ít nhất 5-10 năm kinh ...,Đồng Nai,Bằng cấp: Đại học\nGiới tính: Nữ\nĐộ tuổi: 40 ...,"trưởng phòng hành chính nhân sự, hr administra..."


In [ ]:
pageStart=1 #Trang bắt đầu cào
pageEnd=3 #Trang kết thúc cào
job_urls=crawlJobLinks(pageStart=1,pageEnd=3)

C:\Users\queol\AppData\Local\Temp\ipykernel_28620\3593696083.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path,options=chrome_options)


In [ ]:

df=crawlJobInformation(job_urls)


In [ ]:
df.to_csv("job_information.csv",index=False,encoding="utf-8-sig")